In [4]:
import googlemaps
import pandas as pd
import numpy as np

# with open('key.txt', 'r') as f:
#     key = f.read()[:-1]
gmaps = googlemaps.Client(key='')


In [5]:
import datetime as dt

In [9]:
dt.datetime(2009,1,6,8)

datetime.datetime(2009, 1, 6, 8, 0)

In [12]:
dist_res = gmaps.distance_matrix(origins= (40.721567, -73.991957), 
                                 destinations= (40.695922,-73.993803),
                                 departure_time = dt.datetime(2018,1,2,8),
                                 mode='driving')

In [13]:
dist_res

{'destination_addresses': ['156 Henry St, Brooklyn, NY 11201, USA'],
 'origin_addresses': ['181-183 Chrystie St, New York, NY 10002, USA'],
 'rows': [{'elements': [{'distance': {'text': '4.2 km', 'value': 4208},
     'duration': {'text': '14 mins', 'value': 837},
     'duration_in_traffic': {'text': '13 mins', 'value': 792},
     'status': 'OK'}]}],
 'status': 'OK'}

In [24]:
dist_res['rows'][0]['elements'][0]['duration']['value']

837

In [8]:
df = pd.read_csv('data/gmaps_test.csv')

In [9]:
from tqdm import tqdm_notebook as tqdm

In [10]:
res = []
with tqdm(total=len(df)) as pbar:
    for i, duration, sx, sy, tx, ty in df.itertuples():
        tuesday = np.random.choice([2,9,16,23,30])
        pbar.update(1)
        dist_res = gmaps.distance_matrix(origins= (sx, sy), 
                                         destinations= (tx, ty),
                                         departure_time = dt.datetime(2018,1,tuesday,8),
                                         mode='driving')
        res.append(dist_res)
        if dist_res['status'] != 'OK':
            print('SCREAM')

A Jupyter Widget

KeyboardInterrupt: 

In [11]:
sec = list(map(lambda x : x['rows'][0]['elements'][0]['duration']['value'], res))

In [12]:
sec = np.array(sec)

In [39]:
sec = list(sec)

In [41]:
sec = np.array(sec)

In [13]:
np.save('results/google_errs_with_time.npy', (df['duration'][:len(sec)].as_matrix() - sec))

In [31]:
dist_res

{'destination_addresses': ['156 Henry St, Brooklyn, NY 11201, USA'],
 'origin_addresses': ['181-183 Chrystie St, New York, NY 10002, USA'],
 'rows': [{'elements': [{'distance': {'text': '4.2 km', 'value': 4208},
     'duration': {'text': '14 mins', 'value': 837},
     'status': 'OK'}]}],
 'status': 'OK'}

In [17]:
errs = (df['duration'][:len(sec)] - sec)/60

In [35]:
from json import dump
dump({
    'sd_duration': df['duration'][:len(sec)].std()/60,
    'mean_err' : errs.mean(),
    'sd_err' : errs.std(),
    'mean_abs_err': errs.abs().mean(),
    'median_abs_err': errs.abs().median(),
    '99pct_abs_err' : np.percentile(errs.abs(), 99),
    'test_r2' : 1 - np.std(errs) ** 2 / ((df['duration'].std()/60) ** 2)
}, open('results/google_tues8am.json','w'))

In [31]:
errs.abs().mean()

3.4249263129689154

In [19]:
errs_old = np.load('results/google_errs.npy')

In [26]:
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
pd.Series(errs_old/60 - np.array(errs)[:-10]).value_counts()

 0.000000     2208
-0.250000        6
-0.150000        4
 0.500000        4
 0.050000        3
-0.083333        3
 0.200000        3
-0.350000        3
-0.750000        3
-0.466667        3
 0.400000        2
 0.766667        2
-0.633333        2
 0.150000        2
-2.233333        2
 0.266667        2
 0.733333        2
-0.283333        2
 0.550000        2
 0.666667        2
 0.450000        2
-2.383333        2
-0.066667        2
 4.566667        2
 3.400000        2
 0.016667        2
 0.183333        2
-0.600000        2
-0.216667        2
 0.250000        2
              ... 
 2.800000        1
 1.466667        1
-1.550000        1
-11.466667       1
 0.016667        1
-0.133333        1
 0.600000        1
-0.483333        1
 0.850000        1
-0.200000        1
-0.933333        1
-0.183333        1
 0.650000        1
 1.300000        1
 0.150000        1
 0.650000        1
 0.900000        1
-0.300000        1
 4.183333        1
 2.083333        1
-0.966667        1
-0.133333   

In [23]:
errs_old

array([ -32., -184.,    0., ...,   14.,  -44.,    8.])